In [0]:
%env TF_CPP_MIN_LOG_LEVEL=3
%pip install ydata-synthetic

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GRU, Dense, Conv1D, MaxPooling1D, Flatten,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_log_error
import pyspark as ps
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from ydata_synthetic.synthesizers.timeseries import TimeSeriesSynthesizer
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.stats import bernoulli
from sklearn.model_selection import train_test_split

StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 8, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.3/86.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 70.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 8.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 55.1 MB/s eta 0:00:0000:01:00:01
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.3/325.3 kB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 110.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 192.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 57.9 MB/s eta 0:00:00
  Created wheel for easydict: filename=easydict-1.10-py3-none-any.whl size=6489 sha256=2bb75ee53e95d56c3

In [0]:
df = pd.read_parquet("abfss://ML_Stab_TI_Energinet_EBI@onelake.dfs.fabric.microsoft.com/Ines.Lakehouse/Tables/mw08003_complete_extended")
df.rename(columns = {'Zeitpunkt_x': 'Zeitpunkt'}, inplace=True)
df.sort_values('Zeitpunkt', inplace= True)
# df['temp'] = df['temp'].shift(-8)
# df = df.dropna()
df['Wert'] = df['Wert'].astype(float)

df.reset_index(drop=True, inplace=True)

StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 10, Finished, Available)

In [0]:
# Filter the rows where the year in 'Zeitpunkt' is 2023
df = df[df['Zeitpunkt'].dt.year == 2023]

StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 11, Finished, Available)

In [0]:
def preprocess_for_timegan(df, timestamp_col, num_cols, cat_cols=None, seq_len=24, simulate_bernoulli=False, shuffle=False):
    # Ensure the DataFrame is sorted by timestamp
    df_sorted = df.sort_values(by=timestamp_col).reset_index(drop=True)
    
    # Normalize the specified continuous columns
    scaler = MinMaxScaler()
    df_sorted[num_cols] = scaler.fit_transform(df_sorted[num_cols])
    
    # Initialize cat_cols if None is provided
    if cat_cols is None:
        cat_cols = []
    
    # Simulate each categorical column using Bernoulli distribution if simulate_bernoulli is True
    if simulate_bernoulli and cat_cols:
        for cat_col in cat_cols:
            p = df_sorted[cat_col].mean()  # Probability of success (1) for each column
            df_sorted[cat_col] = bernoulli.rvs(p, size=df_sorted.shape[0])

    # Concatenate column names for continuous and categorical data
    column_names_combined = num_cols + cat_cols

    # Create sequences
    sequences = []
    for i in range(len(df_sorted) - seq_len + 1):
        sequence = df_sorted[column_names_combined].iloc[i:i + seq_len].values
        sequences.append(sequence)

    # Optionally shuffle sequences
    if shuffle:
        random.shuffle(sequences)

    # Convert list of sequences to a 3D numpy array
    sequences_array = np.array(sequences)

    return sequences_array, scaler, column_names_combined

StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 68, Finished, Available)

In [0]:
sequence_length = 24
num_cols = ['Wert', 'temp']
# cat_cols = ['day_type_indicator']

processed_data, scaler, col_names = preprocess_for_timegan(
    df, 
    'Zeitpunkt', 
    num_cols, 
    # cat_cols, 
    seq_len=sequence_length, 
    simulate_bernoulli=True,  # Set to True to simulate binary variable with Bernoulli
    shuffle=False  # Set to True if you want to shuffle the sequences
)

StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 69, Finished, Available)

In [0]:
class TimeGANConfig:
    def __init__(self, column_names, seq_len, hidden_dim, gamma, noise_dim, dim, batch_size, learning_rate, epochs):
        self.seq_len = seq_len
        self.hidden_dim = hidden_dim
        self.gamma = gamma
        self.noise_dim = noise_dim
        self.dim = dim
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.column_names = column_names
        self.n_seq = len(column_names)

        # Store GAN and training arguments
        self.gan_args = self.get_gan_args()
        self.train_args = self.get_train_args()

    
    def get_gan_args(self):
        return ModelParameters(
            batch_size=self.batch_size, 
            lr=self.learning_rate, 
            noise_dim=self.noise_dim, 
            layers_dim=self.dim
        )

    def get_train_args(self):
        return TrainParameters(
            epochs=self.epochs, 
            sequence_length=self.seq_len, 
            number_sequences=self.n_seq
        )
        
    def update_params(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
    
    def initialize_synth(self):
        self.synth = TimeSeriesSynthesizer(modelname='timegan', model_parameters=self.gan_args)

    def fit(self, df):
        self.synth.fit(df, self.train_args, num_cols=self.column_names)


StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 70, Finished, Available)

In [0]:
config = TimeGANConfig(
    column_names=col_names,
    seq_len=sequence_length,
    hidden_dim=24,
    gamma=1,
    noise_dim=32,
    dim=128,
    batch_size=128,
    learning_rate=5e-4,
    epochs=1000
)

# Initialize synthesizer
synthesizer = config.initialize_synth()


# Fit the model
config.fit(df)


StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 71, Finished, Available)

A DataProcessor is not available for the TimeGAN.


Joint networks training: 100%|██████████| 1000/1000 [39:00<00:00,  2.34s/it]


In [0]:
# # Generate synthetic data
syntheitc_data = np.asarray(config.synth.sample(len(processed_data)))

StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 72, Finished, Available)

In [0]:
def create_split_indices(total_samples, test_size, validation_size):
    """
    Create indices for the training, validation, and testing splits.
    """
    indices = np.arange(total_samples)
    np.random.shuffle(indices)  # Shuffle the indices
    test_end = int(total_samples * test_size)
    validation_end = test_end + int(total_samples * validation_size)
    
    test_indices = indices[:test_end]
    validation_indices = indices[test_end:validation_end]
    train_indices = indices[validation_end:]
    return train_indices, validation_indices, test_indices

def split_data_with_indices(processed_data, train_indices, validation_indices, test_indices, target_col_indices):
    """
    Split data into training, validation, and testing sets using provided indices and multiple target columns.
    """
    # Extract features from all timesteps except the last one for the training set
    X_train = processed_data[train_indices, :-1, :]
    
    # Targets are from the last timestep of each sequence for the specified target columns for the training set
    y_train = processed_data[train_indices][:, -1, :][:, target_col_indices]
    
    # Perform similar operations for validation and test sets
    X_validation = processed_data[validation_indices, :-1, :]
    y_validation = processed_data[validation_indices][:, -1, :][:, target_col_indices]
    
    X_test = processed_data[test_indices, :-1, :]
    y_test = processed_data[test_indices][:, -1, :][:, target_col_indices]

    return X_train, X_validation, X_test, y_train, y_validation, y_test

# Assuming processed_data is a numpy array with the shape mentioned earlier
# Assuming you've already defined test_size and validation_size

# Create split indices based on the total number of sequences
total_sequences = processed_data.shape[0]
train_indices, validation_indices, test_indices = create_split_indices(
    total_samples=total_sequences,
    test_size=0.10,
    validation_size=0.15
)

# Assuming target_col_indices = [0, 1] because 'wert' is at index 0 and 'temp' is at index 1
target_col_indices = [0,1]

# Split the data using the indices
X_real_train, X_real_validation, X_real_test, y_real_train, y_real_validation, y_real_test = split_data_with_indices(
    processed_data=processed_data,
    train_indices=train_indices,
    validation_indices=validation_indices,
    test_indices=test_indices,
    target_col_indices=target_col_indices
)

X_synth_train, X_synth_validation, X_synth_test, y_synth_train, y_synth_validation, y_synth_test = split_data_with_indices(
    processed_data=syntheitc_data,
    train_indices=train_indices,
    validation_indices=validation_indices,
    test_indices=test_indices,
    target_col_indices=target_col_indices
)



StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 73, Finished, Available)

In [0]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 74, Finished, Available)

In [0]:
%run models_func

StatementMeta(, , -1, Finished, Available)

In [0]:
#Initialize the model
seq_len = 24
timesteps = seq_len - 1
units = 2
feature_n = 2

real_model = CNN_GRU_regression(timesteps, feature_n, units)

#Train the model on real

history = real_model.fit(
    X_real_train, y_real_train,
    validation_data=(X_real_validation, y_real_validation),
    epochs=100,  # Adjust the number of epochs as needed
    batch_size=128,  # Adjust the batch size as needed
    callbacks=[early_stopping]
)


StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 84, Finished, Available)

2024-03-19:20:18:33,764 WARNING  [tracking_store.py:153] log_inputs not supported
2024-03-19:20:18:33,765 WARNING  [tracking_store.py:153] log_inputs not supported


Epoch 1/100
 1/52 [..............................] - ETA: 1:23 - loss: 0.4090WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0045s vs `on_train_batch_end` time: 0.0070s). Check your callbacks.
Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0045s vs `on_train_batch_end` time: 0.0070s). Check your callbacks.
52/52 [==============================] - 5s 64ms/step - loss: 0.2645 - val_loss: 0.1530
Epoch 2/100
52/52 [==============================] - 0s 6ms/step - loss: 0.1342 - val_loss: 0.1020
Epoch 3/100
52/52 [==============================] - 0s 5ms/step - loss: 0.1030 - val_loss: 0.0848
Epoch 4/100
52/52 [==============================] - 0s 5ms/step - loss: 0.0921 - val_loss: 0.0804
Epoch 5/100
52/52 [=============================

In [0]:
# Initialize the model
synth_model = CNN_GRU_regression(timesteps, feature_n, units)

# Train the model
synth_history = synth_model.fit(
    X_synth_train, y_synth_train,
    validation_data=(X_real_validation, y_real_validation),
    epochs=100,  # Adjust the number of epochs as needed
    batch_size=128,  # Adjust the batch size as needed
    callbacks=[early_stopping]
)


StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 85, Finished, Available)

2024-03-19:20:19:17,294 WARNING  [tracking_store.py:153] log_inputs not supported
2024-03-19:20:19:17,295 WARNING  [tracking_store.py:153] log_inputs not supported


Epoch 1/100
 1/52 [..............................] - ETA: 1:22 - loss: 0.4547WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0046s vs `on_train_batch_end` time: 0.0070s). Check your callbacks.
Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0046s vs `on_train_batch_end` time: 0.0070s). Check your callbacks.
52/52 [==============================] - 3s 25ms/step - loss: 0.3075 - val_loss: 0.2167
Epoch 2/100
52/52 [==============================] - 0s 5ms/step - loss: 0.1552 - val_loss: 0.1531
Epoch 3/100
52/52 [==============================] - 0s 5ms/step - loss: 0.1412 - val_loss: 0.1514
Epoch 4/100
52/52 [==============================] - 0s 5ms/step - loss: 0.1386 - val_loss: 0.1525
Epoch 5/100
52/52 [=============================

In [0]:
# Generate predictions for CNN_GRU_regression
print('for real')
real_pred = real_model.predict(X_real_test)
print('for synth')
synthetic_pred = synth_model.predict(X_real_test)

StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 86, Finished, Available)

for real
28/28 [==============================] - 0s 1ms/step
for synth
28/28 [==============================] - 0s 1ms/step


In [0]:
%run Visualizations_func

StatementMeta(, , -1, Finished, Available)

In [0]:
print('compared with synthetic')
print(calculate_metrics_2(y_real_test, real_pred, synthetic_pred))

StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 96, Finished, Available)

compared with synthetic
     Feature  R2 Score Real  R2 Score Synth  MAE Real  MAE Synth
0  Feature_0       0.555274       -0.009890  0.059949   0.101780
1  Feature_1       0.935634       -0.642182  0.040584   0.211118


In [0]:
from sklearn.metrics import pairwise_distances
def calculate_mmd(X, Y, gamma=None):
    """
    Calculate the Maximum Mean Discrepancy (MMD) between two samples, X and Y.
    The kernel can be computed using a Gaussian kernel with bandwidth gamma.
    If gamma is None, it uses the median heuristic.
    """
    n = X.shape[0]
    m = Y.shape[0]
    
    # Compute the pairwise distances in the Gaussian kernel
    XX = pairwise_distances(X.reshape(n, -1), metric='euclidean')
    YY = pairwise_distances(Y.reshape(m, -1), metric='euclidean')
    XY = pairwise_distances(X.reshape(n, -1), Y.reshape(m, -1), metric='euclidean')
    
    if gamma is None:
        # Median heuristic for the bandwidth
        gamma = np.median(np.hstack((XX.flatten(), YY.flatten(), XY.flatten())))
    
    # Compute the kernel matrices
    K_XX = np.exp(-gamma * XX)
    K_YY = np.exp(-gamma * YY)
    K_XY = np.exp(-gamma * XY)
    
    # Compute the MMD
    mmd = K_XX.mean() + K_YY.mean() - 2 * K_XY.mean()
    return mmd

# Compute MMD between the real and synthetic datasets
mmd_value = calculate_mmd(processed_data, syntheitc_data)
print(f"MMD: {mmd_value}")

StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 97, Finished, Available)

MMD: 0.04983536586193418


In [0]:
# Calculate Pearson correlations
pearson_corrs_synthetic = calculate_pearson_correlations(
    processed_data, syntheitc_data
)

print(f'The pearson correlation for synthetic data {pearson_corrs_synthetic}')

StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 98, Finished, Available)

The pearson correlation for synthetic data [-0.0016056769741046215, 0.0009930196355194447]


In [0]:
from scipy.stats import ks_2samp
from scipy.spatial.distance import cityblock

def calculate_ks_test(sample1, sample2):
    # Perform the KS test
    ks_statistic, p_value = ks_2samp(sample1, sample2)
    return ks_statistic, p_value

def calculate_tvd(p, q):
    # The TVD is half the L1-norm (Manhattan distance) between two probability distributions
    tvd = cityblock(p, q) / 2.0
    return tvd

# Assuming your datasets are named 'processed_data' and 'synthetic_data'
# with shape (8737, 24, 2)
# Let's loop over each attribute

num_attributes = processed_data.shape[2]  # In your case, 2

# Store the results
ks_results = []
tvd_results = []

for attribute_index in range(num_attributes):
    # Extracting the attribute across all sequences for both datasets
    real_attribute_data = processed_data[:, :, attribute_index].flatten()
    synthetic_attribute_data = syntheitc_data[:, :, attribute_index].flatten()
    
    # KS Test for the current attribute
    ks_statistic, p_value = calculate_ks_test(real_attribute_data, synthetic_attribute_data)
    ks_results.append((ks_statistic, p_value))
    
    # TVD for the current attribute
    # Get the histograms as probability distributions for each attribute
    p = np.histogram(real_attribute_data, bins=50, range=(0, 1), density=True)[0]
    q = np.histogram(synthetic_attribute_data, bins=50, range=(0, 1), density=True)[0]
    
    tvd = calculate_tvd(p, q)
    tvd_results.append(tvd)

# Now, ks_results and tvd_results hold the KS statistics/p-values and TVDs for each attribute
for i, (ks, tvd) in enumerate(zip(ks_results, tvd_results)):
    print(f"Attribute {i}: KS statistic: {ks[0]}, P-value: {ks[1]}, TVD: {tvd}")


StatementMeta(, f5af308f-1c3d-48b7-b37e-6b100cb697b1, 99, Finished, Available)

Attribute 0: KS statistic: 0.057928922971271606, P-value: 2.882655594196529e-306, TVD: 5.880640972897213
Attribute 1: KS statistic: 0.10455057037121818, P-value: 0.0, TVD: 6.8804558189807805
